# Deep Learning

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- Conv Net(s)
- ResNets / HighwayNets - RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.

----

In [32]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
import os

In [29]:
print(tf.__version__)
print(tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

2.6.0
/device:GPU:0


2021-12-13 16:05:07.941920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-13 16:05:07.942006: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Importation du dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [5]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = tf.keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

In [6]:
LOG_DIR = os.path.join("logs")

## Fixer les seeds

In [33]:
tf.random.set_seed(42)
np.random.seed(42)

# Modèle linéaire

In [7]:
KERNEL_INITIALIZER = 'zeros'
OPTIMIZER = tf.keras.optimizers.SGD()
BATCH_SIZE = 256
EPOCHS = 10
SHUFFLE = True
LINEAR_LOG = os.path.join(LOG_DIR,
                       "linear",
                       f"linear_model_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_{type(OPTIMIZER).__name__}_ki_{KERNEL_INITIALIZER}")

In [8]:
linear_model = tf.keras.models.Sequential()

linear_model.add(tf.keras.layers.Flatten())
linear_model.add(tf.keras.layers.Dense(IMG_SIZE[0]*IMG_SIZE[1]*IMG_SIZE[2],
                                       activation='relu'))
linear_model.add(tf.keras.layers.Dense(NUM_CLASSES,
                                       activation='softmax'))

2021-12-13 15:13:20.565990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-13 15:13:20.566008: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
linear_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                     optimizer=OPTIMIZER,
                     metrics="accuracy")

In [10]:
linear_model.fit(x_train,
                 y_train,
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 validation_data=(x_test, y_test),
                 shuffle=SHUFFLE,
                 callbacks=[tf.keras.callbacks.TensorBoard(LINEAR_LOG, histogram_freq=1)])

2021-12-13 15:13:20.653091: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 15:13:20.653099: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 15:13:20.653133: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 15:13:21.544632: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-13 15:13:21.546659: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-13 15:13:21.744502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
 11/196 [>.............................] - ETA: 2s - loss: 2.2524 - accuracy: 0.1573

2021-12-13 15:13:22.324818: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 15:13:22.324826: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 15:13:22.341655: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 15:13:22.343940: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 15:13:22.347842: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/linear/linear_model_ep_10_bs_256_opt_SGD_ki_zeros/train/plugins/profile/2021_12_13_15_13_22

2021-12-13 15:13:22.348537: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/linear/linear_model_ep_10_bs_256_opt_SGD_ki_zeros/train/plugins/profile/2021_12_13_15_13_22/MacBook-Pro-de-Reda.local.trace.json.gz
2021-12-13 15:13:22.350591: I tensorflow/core/profiler/rpc/client/save_

196/196 [==============================] - ETA: 0s - loss: 1.9865 - accuracy: 0.2939

2021-12-13 15:13:24.987170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


196/196 [==============================] - 4s 16ms/step - loss: 1.9865 - accuracy: 0.2939 - val_loss: 1.9341 - val_accuracy: 0.3270
Epoch 2/10
196/196 [==============================] - 3s 14ms/step - loss: 1.8337 - accuracy: 0.3618 - val_loss: 1.8417 - val_accuracy: 0.3427
Epoch 3/10
196/196 [==============================] - 3s 13ms/step - loss: 1.7792 - accuracy: 0.3807 - val_loss: 1.7670 - val_accuracy: 0.3859
Epoch 4/10
196/196 [==============================] - 3s 13ms/step - loss: 1.7414 - accuracy: 0.3982 - val_loss: 1.7650 - val_accuracy: 0.3812
Epoch 5/10
196/196 [==============================] - 3s 13ms/step - loss: 1.7158 - accuracy: 0.4070 - val_loss: 1.7581 - val_accuracy: 0.3789
Epoch 6/10
196/196 [==============================] - 3s 13ms/step - loss: 1.6906 - accuracy: 0.4173 - val_loss: 1.7275 - val_accuracy: 0.4045
Epoch 7/10
196/196 [==============================] - 3s 13ms/step - loss: 1.6705 - accuracy: 0.4249 - val_loss: 1.6872 - val_accuracy: 0.4107
Epoch 8/10

In [11]:
scores = linear_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 2s 5ms/step - loss: 1.6627 - accuracy: 0.4235
Test loss: 1.6626532077789307
Test accuracy: 0.42350003123283386


In [12]:
# Ajouter une matrice de correlation

## MLP

Hyperparameter tuning using Tensorflow and Tensorboard : https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

In [58]:
NUM_LAYERS = hp.HParam('num_layers', hp.Discrete(range(2, 6)))
NUM_UNITS = hp.HParam('num_units', hp.Discrete([8, 16, 32, 64, 128]))
OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))
BATCH_SIZE = 256
EPOCHS = 200
SHUFFLE = True
HPARAM_DIR = os.path.join(LOG_DIR, "mlp", "hparam_tuning")

In [59]:
with tf.summary.create_file_writer(HPARAM_DIR).as_default():
    hp.hparams_config(
    hparams=[NUM_UNITS, NUM_LAYERS, OPTIMIZER],
    metrics=[hp.Metric("accuracy", display_name='Accuracy')],
  )


In [63]:
def mlp_model(hparams: hp) -> tf.keras.models.Model:
    random_units = []
    num_layer = None
    input_ = tf.keras.layers.Input(shape=(32, 32, 3))
    hidden_layers = tf.keras.layers.Flatten()(input_)
    for opt in OPTIMIZER.domain.values:
        for num_layers in NUM_LAYERS.domain.values:
            random_unit = np.random.choice(NUM_UNITS.domain.values, 1)
            random_units.append(random_unit)
            hidden_layers = tf.keras.layers.Dense(random_unit, activation="relu")(hidden_layers)

        output_ = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(hidden_layers)
        mlp = tf.keras.models.Model(input_, output_)
        mlp.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=opt,
                    metrics="accuracy")
        MLP_LOG = os.path.join(LOG_DIR, "mlp", "hparam_tuning",
                f"mlp_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_{type(opt).__name__}_layers_{NUM_LAYERS}")
        mlp.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(MLP_LOG, histogram_freq=1),
                           hp.KerasCallback(MLP_LOG, hparams)]
               )
        _, accuracy = mlp.evaluate(x_test, y_test)
    return history

2021-12-13 17:08:59.442779: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 17:08:59.442796: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 17:08:59.442870: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


TypeError: hparams() got an unexpected keyword argument 'metrics'

2021-12-13 17:00:51.640119: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 17:00:51.640141: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 17:00:51.640165: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


NameError: name 'hparams' is not defined